## The Importance of Cell Connectivity for Efficient and Accurate Modeling of Flow through Dipping Aquifers with MODFLOW 6

Alden M. Provost, Kerry Bardot, Christian D. Langevin, James L. McCallum (2024)

In [ ]:
%run ./routines.ipynb      # Associated notebook with necessary functions
%run ./figures.ipynb       # Associated notebook to generate figures for manuscript
ws = './disu_model'

domain = True              # option to include "domain" (True) or not (False)
spdis_recalc = True        # option to recalculate specific discharge within the notebook
dztol = 0.00001            # tolerance for determining overlap (dz > dztol) in a vertically staggered grid

In [ ]:
# scenario parameters
# scenario names -- keys of parameters dictionary are scenario names
#                   built automatically from scenario options
# staggered -- vertically staggered grid, i.e., cross-connections (True)
#              or not, i.e., vertically offset (False)
# xt3d -- xt3d on (True) or off, i.e., standard formulation (False)

# scenario options
options = [
    {
        "optionname": "staggered",
        "optionstrings": {
            False: "-vo",
            True: "-vs",
        },
    },
    {
        "optionname": "xt3d",
        "optionstrings": {
            False: "-s",
            True: "-x",
        },
    },
]

# build parameters dictionary
parameters = build_parameters(options)

In [ ]:
# Lx is the full width of the model. The grid spacing in the x direction is
# delr = Lx / ncol, and so the horizontal distance between the left and right
# boundary conditions is Lx - delr. The cells in the channel are square, i.e.,
# delz_chan = delr. The cross-sectional model has unit thickness "into the page,"
# i.e., nrow = 1 and delc = 1.

# The full thickness of the channel measured vertically is nlay_chan * delz_chan.
# Theta is the angle (deg) at which the channel slopes, and so the width of the
# "true" channel measured perpendicularly to the slope, i.e., the width available 
# for flow along the slope of the channel, is nlay_chan * delz_chan * cos(theta).
# The increase in elevation (vertical offset) as one moves a column to the right
# within the channel is delr * tan(theta).

# adjustable grid parameters
nlay_chan = 3                         # number of layers in channel
ncol = 11                             # number of columns in model
Lx = 11.0                             # full horizontal width of model
theta = 30.                           # angle of inclination of channel
k_dom = 1e-6                          # domain conductivity
anisotropic = False

# set up dis grid and conductivities
mgs, nlay_dom_upper, icelltype, delz_chan, zspan, zthick, zthick_dom_ll, thick_arr \
    = set_up_dis_grid(nlay_chan, ncol, Lx, theta, domain)
cond = set_up_cond(k_dom)

# Calculate volumetric flow Analytical = kai 
flow_analytical = 1 * zthick * math.cos(math.radians(theta)) * 1

# arrays for storing results
models_all, head_all, spdismf6_all, q_all, qx_all, qy_all, qz_all, qnface_all, flow_all = [], [], [], [], [], [], [], [], []

# loop over scenarios
fig = plt.figure(figsize = (8,10))#, constrained_layout=True)
nscen = len(parameters)
for i in range(nscen):
    # convert to disu grid
    d2d = convert_to_disu(mgs, i, dztol)
    xface, yface, zface, ifacetype = get_face_info(d2d)
    # run scenario and store results
    gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error = simulation(i)
    models_all.append(gwf), head_all.append(head), spdismf6_all.append(spdismf6)
    qx_all.append(qx), qy_all.append(qy), qz_all.append(qz)
    q_all.append((qmagmid, qangmid, flow_error))
    qnface_all.append(qnface)
    flow_all.append(flow)
    extent = (0., Lx, zthick_dom_ll, zthick_dom_ll + zspan + delz_chan)
    #extent = None
    #dxdysin = float(Lx / ncol) * 1. * math.sin(theta * math.pi / 180.)
    scale_bndy = 10. ### * 0.5 * (1. + k_dom)
    head_diff = head - head_analyt
    print("Max head error = ", head_diff.max())
    fig2(i, vmin = -20, vmax = 0, extent=extent,
    scale_chan=10., scale_bndy=scale_bndy, scale_dom=10., normalize=False)
    plt.tight_layout
fig.savefig('../figures/fig2_paper.png' , dpi = 450)

In [ ]:
table1()

### Investigating adding model layers within hydrogeologic layers 

In [ ]:
ncol = 11                             # number of columns in model
Lx = 11.0                             # full horizontal width of model
theta = 30.                           # angle of inclination of channel
k_dom = 1.e-6                         # domain conductivity

# grid resolution cases
resolution_scenarios = np.arange(1,10,1)
nr = len(resolution_scenarios)

res_results = [] 

# loop over grid resolution cases
for n in range(nr):
    nlay_chan = resolution_scenarios[n]   # number of layers in channel
    # set up dis grid and conductivities
    mgs, nlay_dom_upper, icelltype, delz_chan, zspan, zthick, zthick_dom_ll, thick_arr \
        = set_up_dis_grid(nlay_chan, ncol, Lx, theta, domain)
    cond = set_up_cond(k_dom)
    
    # Calculate volumetric flow Analytical = kai 
    flow_analytical = 1 * zthick * math.cos(math.radians(theta)) * 1    
    #print(zthick)
    results = []
    for i in range(nscen):
        # convert to disu grid
        d2d = convert_to_disu(mgs, i, dztol)
        xface, yface, zface, ifacetype = get_face_info(d2d)
        # run scenario and store results
        gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error = simulation(i)  
        results.append((qmagmid, qangmid, flow_error))
    res_results.append(results)  
fig3_results = np.array(res_results)

In [ ]:
fig3()

### Investigating K contrast between hydrogeologic layer and domain

In [ ]:
nlay_chan = 3                         # number of layers in channel
ncol = 11                             # number of columns in model
Lx = 11.0                             # full horizontal width of model

# dip and K contrast cases
dip = np.arange(0, 75, 2.5)   # For nice contours use (0,80,1)
#log = np.arange(0, 3, 0.5)    # For nice contours use (0,4,0.2)
log = [0, np.log10(2), np.log10(5), 1, 2]
#log = [0.25, 0.5, 1, 2]
Kcontrast = [10**i for i in log]
Kcontrast = np.round(Kcontrast, 0)
# arrays for storing results
qmag_results = np.zeros((4, len(Kcontrast), len(dip)))
qang_results = np.zeros((4, len(Kcontrast), len(dip)))
dip_array = np.zeros((4, len(Kcontrast), len(dip)))
Kcontrast_array = np.zeros((4, len(Kcontrast), len(dip)))

# loop over scenarios
for s in range(len(parameters)):
    # loop over dip cases
    for d in range(len(dip)):
        theta = dip[d]                        # angle of inclination of channel
        # set up dis grid
        mgs, nlay_dom_upper, icelltype, delz_chan, zspan, zthick, zthick_dom_ll, thick_arr \
            = set_up_dis_grid(nlay_chan, ncol, Lx, theta, domain)
        d2d = convert_to_disu(mgs, s, dztol)
        xface, yface, zface, ifacetype = get_face_info(d2d)
        # loop over K contrast cases
        for kc in range(len(Kcontrast)):
            #print('sim = %i, dip = %f0.1, contrast = %f0.2' %(s, dip[d], Kcontrast[kc]))
            k_dom = 1./Kcontrast[kc]              # domain conductivity
            # set up conductivities
            cond = set_up_cond(k_dom)
            # run case and store results
            gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error = simulation(s)
            qmag_results[s][kc][d] = qmagmid
            qang_results[s][kc][d] = qangmid  
            Kcontrast_array[s][kc][d] = Kcontrast[kc]
            dip_array[s][kc][d] = dip[d]

In [ ]:
## Finding the maximum errors for ranging dip and contrast
#s = 3 #scenario [0,1,2,3]
#kc = 1 # kcontrast [  1.,   2.,   5.,  10., 100.]
#print('Flux mag')
#for i in range(len(dip_array[scenario][kc])):
#    print(dip_array[s][kc][i], qmag_results[s][kc][i])
#print('\nFlux angle')
#for i in range(len(dip_array[scenario][kc])):
#    print(dip_array[s][kc][i], qang_results[s][kc][i])

In [ ]:
fig4()

In [ ]:
# ANISOTROPY TEST
nlay_chan = 3
ncol = 11                             # number of columns in model
Lx = 11.0                             # full horizontal width of model
theta = 30.                           # angle of inclination of channel
k_dom = 1.e-6                         # domain conductivity
dip = 30
anisotropic = True

anisotropic_scenarios = [1,10,100,1000,10000]
nr = len(anisotropic_scenarios)

ani_results = [] 

# loop over grid resolution cases
for n in range(nr):
    ratio = anisotropic_scenarios[n]
    #print(ratio)
    
    # set up dis grid and conductivities
    mgs, nlay_dom_upper, icelltype, delz_chan, zspan, zthick, zthick_dom_ll, thick_arr \
        = set_up_dis_grid(nlay_chan, ncol, Lx, theta, domain)
    cond = set_up_cond(k_dom)
    
    # Calculate volumetric flow Analytical = kai 
    flow_analytical = 1 * zthick * math.cos(math.radians(theta)) * 1    
    
    results = []
    for i in range(nscen):
        # convert to disu grid
        d2d = convert_to_disu(mgs, i, dztol)
        xface, yface, zface, ifacetype = get_face_info(d2d)
        # run scenario and store results
        gwf, head, spdismf6, qx, qy, qz, qmagmid, qangmid, qnface, flow, flow_error = simulation(i)  
        results.append((qmagmid, qangmid, flow_error))
    ani_results.append(results)  
fig5_results = np.array(ani_results)

In [ ]:
fig5()